<h1>
   DataTweet
</h1>

<br>
<div>
    AUTOR:Luis Enrique Seijas Salomon<br>
    FECHA: Septiembre 2021<br>
</div>

<hr>

### DataTweet

Es una aplicación que extrae información a tiempo real de Twitter por medio del Api que Twitter tiene a disposición. La idea de esta aplicación es realizar la extracción de datos de interés sin necesidad de tener conocimientos sobre el fruncimiento de api de Twitter, no grande conocimientos como realizar conexiones y peticiones GET y POST en protocolos HTTP.

Esta aplicación provee una capa de abstracción, que, con seguir una serie de pasos ya podemos extraer datos de interés de la plataforma sin necesidad de ser expertos en el tema. 

Los detalles se pueden consultar en la documentación asociada al repositorio.


### Librerías y paquetes a instalar


In [ ]:
!pip install emoji_extractor
!pip install emoji
import nltk
nltk.download('stopwords')
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install jupyter-dash
!pip install dash-bootstrap-components

### Lista de librerías implementadas

Para el desarrollo de la herramienta se realizó en lenguaje programación Python, bajo el entorno de desarrollo, júpiter notebook.  En dicho desarrollo se hicieron usa de múltiples Liberia de Python como :

- Pandas: Usada para creación y exportación de las tablas donde se almacena los datos extraídos.
- Reguests: Usada para establecer la comunicación entre la herramienta y el API de Twitter mediante peticiones HTTP (GET y POST).
- NLTK,Spacy,en_core_web_sm,emoji_extractor: estas Liberia se usaron para un procesado previo de los datos antes de exportar el data set final, el cual consiste en separar emojis en casa de que existan, así como analizar el grado de neutralidad, negatividad y positividad de los mismos, también un proceso de eliminación de signos de puntuación, separación de cada palabras y lematización de las mismas, con la finalidad de aportar metadatos de los datos recolectados para procesos de análisis posteriores.  
- Dash: Usada para implementar la interfaz gráfica en lenguaje HTML mediante lenguaje Python, esta herramienta permite la visualización dinámica de datos. En esta proyectó se implementó esta librería para la creación de una pequeña interfaz de usuario para facilitar el uso de la herramienta. Conjuntamente con esta librería se usaron otros paquetes como: dash_bootstrap_components, para la implementación de estilos gráficos y jupyter_dash para implementación de la interfaz desde el entorno de desarrollo júpiter notebook


In [29]:
import os
import csv
import json
import requests
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import seaborn as sns
import re
from emoji_extractor.extract import Extractor
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import es_core_news_sm

import plotly.express as px
import dash
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL


### Api Twitter

Se establecen los parámetros de configuración para la comunicación con el API de Twitter, los endpion usado son los de extracción de datos a tiempo real.

Parámetros de configuración del API Twitter (Endpoint)

Se puede consultar documentación en: https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/api-reference/get-tweets-search-stream


In [2]:
stream_url="https://api.twitter.com/2/tweets/search/stream"
rules_url="https://api.twitter.com/2/tweets/search/stream/rules"
user_url="https://api.twitter.com/2/users"

### Funciones de conectividad y extraccion de datos con el API de Twitter

Las reglas de extracción de datos son un conjunto de parámetros mediante el cual nos comunicamos con el API para obtener la mayor cantidad de tweet de nuestro interés, actualmente existen múltiples reglas para definir la búsqueda y extracción de los datos, las cuales se rigen bajo una sintaxis especifica establecida en la documentación oficial de Twitter.

Debido a que las reglas se rigen por una sintaxis definida, las cuáles no son del todo sencillas, por lo cual,  se crearon un conjunto de funciones que facilitan la implementación de las mismas.

Dentro de las funciones que facilitan la creación de reglas se encuentran:

In [3]:
def GuardarToken(token,user=""):
    """
        Esta función almacena las credenciales del usuario un fichero .json
        Parámetros:
          token: string
            Token de validación
          user: string
              Nombre de usuario para comprobar la validez de token
        ----------------------------------------------------------------------------------   
        Return
          No retorna nada
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> GuardarToken("xxxxxxxxx...xxxxxxxxxxxx...",user="@nombreUser")
    """
    jsonFile = open("Credenciales.json", "w")
    jsonFile.write('{')
    jsonFile.write('"user": "'+user+'",')
    jsonFile.write('"token": "'+token+'"')
    jsonFile.write('}')
    jsonFile.close()
            
def bearer_oauth(r):
    """
        Esta función permite la autenticación con el API de Twitter
        Parámetros:
          r: request
    """
    with open("Credenciales.json") as tokenfile:
        tokenData = json.load(tokenfile)
        r.headers["Authorization"] = f'Bearer {tokenData["token"]}'
        r.headers["User-Agent"] = "ExtractorData"
        return r

def crearReglas(rules_value, tag_value):
    """
        Esta función permite crear las reglas, de tal forma que puedan ser interpretadas 
        por el API de Twitter
        Parámetros:
          rules_value: string
            La cadena con todos los parámetros a formar la regla de búsqueda
          tag_value: string
              Candena con el nombre identificador de la regla
        ----------------------------------------------------------------------------------   
        Return
          Crear la regla directamente en la sesión abierta del API de Twitter para el token indicado
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> crearReglas("(palabra)(from:nombreuser)", "NombreRegla")
              Busca tweets que contengan la palabra "palabra" y sean emitidos por el usuario "nombreuser"
    """
    sample_rules = [{"value": rules_value,"tag": tag_value}]
    payload = {"add": sample_rules}
    response = requests.post(rules_url,auth=bearer_oauth,json=payload)
    if response.status_code != 201:
        raise Exception(
            "Error al crear la regla (HTTP {}): {}".format(response.status_code, response.text)
        )

def obtenerReglas():
    """
        Esta función obtiene las reglas actuales cargadas en la sesión del API
        Return
          Diccionario de reglas
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> obtenerReglas()
    """
    response = requests.get(rules_url, auth=bearer_oauth)
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    return response.json()

def borrarReglas(rules):
    """
        Esta función borra todas las reglas actuales cargadas en la sesión del API
        Parámetros:
          rules: diccionario
            Reglas cargas en la sesión de Twitter
        ----------------------------------------------------------------------------------   
        Return
          No retorna nada
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> borrarReglas(obtenerReglas())
    """
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(rules_url,auth=bearer_oauth,json=payload)
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
        
def validarUsuario(user):
    """
        Esta función valida que el usuario introducido sea válido y exista en Twitter
        Parámetros:
          user: string
            Nombre de usuario puede ser con @ o sin ella.
        ----------------------------------------------------------------------------------   
        Return
           Si existe o no el usuario : bool
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> validarUsuario("@nombreuser")
              True
    """
    usernames = f"usernames={user}"
    user_fields = "user.fields=id,created_at"  
    response = requests.request("GET", "{}/by?{}&{}".format(user_url,usernames,user_fields), auth=bearer_oauth,)
    if response.status_code != 200:
        return False
    else:
        if "data" in response.json():
            return True
        else:
            return False

def tweetUsuario(listaUsuario):
    """
        Esta función formatea los nombres de usuarios para que el API de Twitter entienda
        que se trata de un usuario a incluir en los criterios de búsqueda
        Parámetros:
          listaUsuario: List
            Lista de usuarios
        ----------------------------------------------------------------------------------   
        Return
           queryUsuarios: List<string>
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> tweetUsuario(["@nombreuser","@nombreuser2"])
                  ["from:nombreuser","from:nombreuser2"]
    """
    queryUsuarios = []
    for user in listaUsuario:
        if "@" in user:
            user = user.replace("@","")
        if validarUsuario(user):
            queryUsuarios.extend([f"from:{user}"])
    return queryUsuarios

def tweetIdioma(idioma):
    """
        Esta función formatea el idioma indicado para que el API de Twitter entienda
        que se solo va a buscar tweets que sean emitidos en ese idioma

        Parámetros:
          idioma: string
            idioma del tweet ("es","en")
        ----------------------------------------------------------------------------------   
        Return
           queryIdiomas: List<string>
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> queryIdiomas(["es"])
                  ["lang: es"]
    """
    queryIdiomas=""
    listaIdiomasPermitidos = ["es","en"] # esta lista se puede cargar por configuración
    if idioma in listaIdiomasPermitidos:
        queryIdiomas="lang:{}".format(idioma)        
    return queryIdiomas

def tweetExcluir(valor):
    """
        Esta función es de propósito general formatea cualquier parámetro para negarlo y que 
        sea omitido en la búsqueda de Tweet
        Parámetros:
          valor: string
            Cadena de texto de cualquier parámetro a omitir
        ----------------------------------------------------------------------------------   
        Return
           string
                Retorna la negación formateada
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> tweetExcluir("from:nombreuser")
                  -from:nombreuser
                  En este caso se le pasa como parámetro un nombre de usuario y se niega, esto es interpretado en 
                  el api como : extraer todos los tweets que no sean de este usuario
    """
    return "-{}".format(valor)

def tweetRetweet(opcion=True):
    """
        Esta función permite obtener Retweet o no
        Parámetros:
          opcion: bool
            Si se desea o no retweet en la búsqueda, por defecto viene a true
            (True -> Estraer tweet y retweet | False -> Solo Tweet)
        ----------------------------------------------------------------------------------   
        Return
           retweet: string
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> tweetRetweet(opcion=false)
                  -is:retweet
    """
    retweet="is:retweet"
    if opcion == True:
        return retweet
    else:
        return tweetExcluir(retweet)
    
def excluirPalabras(listaPalabras):
    """
        Esta función excluye una lista de palabras
        Parámetros:
          listaPalabras: list<string>
            Lista de palabras a excluir en la búsqueda
        ----------------------------------------------------------------------------------   
        Return
           queryPalabrasExcluidas: list<string>
               lista de palabras formateadas a ser excluidas en la búsqueda
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> excluirPalabras(["palabra1","palabra2"])
                  ["-palabra1","-palabra2"]
    """
    contadorPalabras=0
    queryPalabrasExcluidas = ""
    for palabra in listaPalabras:
        if " " in palabra:
            #Es una frase, hay que exlcuir la frase completa
            conjuntoPalabras="({})".format(palabra)
            queryPalabrasExcluidas+="{}".format(tweetExcluir(conjuntoPalabras))
        else:    
            queryPalabrasExcluidas+="{}".format(tweetExcluir(palabra))
        if contadorPalabras < len(listaPalabras)-1:
            queryPalabrasExcluidas+=" "
            contadorPalabras+=1
            
    return queryPalabrasExcluidas

def agruparPalabras(cadenaPalabras):
    """
        Esta función agrupa palabras para ser búsquedas como grupos, se usa frecuente para buscar frases
        Parámetros:
          cadenaPalabras: string
            Cadena de palabras
        ----------------------------------------------------------------------------------   
        Return
           string
               Cadena de palabras agrupadas
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> agruparPalabras("palabra1 palabra2")
                  ("palabra1 palabra2")
    """
    return "({})".format(cadenaPalabras)


def incluirPalabras(listaPalabras,logica=" "): #logica AND o OR
    """
        Esta función Permite extracción de Tweets que tengan un con juntos de palabras o 
        frases determinas, para esto, se recibe como parámetro una lista de palabras y 
        una opción lógica (AND o OR,) lo que permite buscar tweet que tengan todas las palabras
        indicadas (AND) o que contengan al menos una de las palabras indicadas (OR).
        Parámetros:  
          listaPalabras: list<string>
            lista de palabras a incluir
            logica : bool
                AND - tweet que contengan todas las palabras indicadas.
                OR  - tweet que contengan alemanes una de las palabras indicadas.
                " " - si va vacío se interpreta como un OR
        ----------------------------------------------------------------------------------   
        Return
           queryPalabras : list<string>
               Lista de palabras
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> incluirPalabras(["palabra1","palabra2"],logica="OR")
                  ["palabra1 OR palabra2"]
    """
    queryPalabras = ""
    contadorPalabras=0
    if logica =="OR" or logica ==" ":
        logica= " OR "
        for palabra in listaPalabras:
            queryPalabras+="{}".format(palabra)
            if contadorPalabras < len(listaPalabras)-1:
                queryPalabras+="{}".format(logica)
                contadorPalabras+=1
        return agruparPalabras(queryPalabras)
    elif logica =="AND":
        for palabra in listaPalabras:
            queryPalabras+="{}".format(agruparPalabras(palabra))
        return queryPalabras

def obtenerTiempo(dias=0, minutos=0):
    """
        Esta función obtiene el tiempo actual del sistema o el indicado, 
        se encarga obtener el valor de timeout y los tiempos de ejecución de la herramienta
        
        Parámetros:  
          dias: int
            Número  de dias
          minutos: int
            Número  de minutos
         
        ----------------------------------------------------------------------------------   
        Return
           time : datatime
               Tiempo
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> obtenerTiempo(dias=0, minutos=0)
                  tiempo actual del sistema
    """
    sumarTiempo=timedelta(0)
    if dias!=0:
        sumarTiempo=timedelta(dias)
    if minutos!=0:
        sumarTiempo=timedelta(minutes=minutos)
    time = datetime.now()+sumarTiempo
    return time

def obtenerTweets(cantTweets=0,minutos=0,dias=0,timeoutMin=10):
    """
        Esta función engloba todas las funciones, establece la comunican con el API para iniciar la extracción de Tweets
                
        Parámetros:  
          cantTweets: int
              Cantidad de tweets que se desean extraer
          minutos: int
            Minutos que desean extraer Tweet
          dias
            Número de días que se desean estar extrayendo Tweet
            timeoutMin: int
                En caso de no encontrar ningún Tweet y si se le indica el parámetro de cantidad, el 
                programa finaliza la ejecución en el tiempo indicado, por defecto 10 min.
        ----------------------------------------------------------------------------------   
        Return
           bool
               Si el proceso termino correctamente o no
            fichero json
                Crea un fichero json con los datos del todos los tweets extraídos.
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> obtenerTweets(cantTweets=0,minutos=0,dias=0,timeoutMin=10)
                  tiempo actual del sistema
    """
    tiempoSalida=obtenerTiempo(dias=0,minutos=timeoutMin)
    jsonFile = open("DataTweet.json", "w")
    jsonFile.write("[")
    DataDic={}
    if cantTweets == 0 and minutos== 0 and dias==0:
        cantTweets=100
    auxCantTweets=0
    tiempoFinalizacion=0
    if minutos!=0 and dias==0:
        tiempoFinalizacion=obtenerTiempo(minutos=minutos)
    if dias!=0 and minutos==0:
        tiempoFinalizacion=obtenerTiempo(dias=dias)
    
    tiempo=obtenerTiempo(minutos=minutos)
    response = requests.get(stream_url, auth=bearer_oauth, stream=True,)
    #print(response.status_code)
    if response.status_code != 200:
        print("error")
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            jsonWrite=""
            json_response = json.loads(response_line)
            jsonWrite=json.dumps(json_response)
            if jsonWrite !="":
                #print("response_line:"+json.dumps(json_response))
                jsonFile.write(json.dumps(json_response))
                if cantTweets != 0 and minutos == 0 and dias==0:
                    if auxCantTweets < cantTweets-1:
                        auxCantTweets+=1
                        jsonFile.write(",")
                        #print(json.dumps(json_response["data"]["text"], indent=4, sort_keys=True))
                        #print("\n")
                    else:
                        jsonFile.write("]")
                        jsonFile.close()
                        return True
                    
                if minutos !=0 or dias!=0:
                    if tiempoFinalizacion < obtenerTiempo():
                        jsonFile.write("]")
                        jsonFile.close()
                        return True
                    else:
                        jsonFile.write(",")
                if obtenerTiempo() > tiempoSalida and cantTweets != 0 and (minutos ==0 or dias==0):
                    jsonFile.write("]")
                    jsonFile.close()
                    return False


 



### Funciones de procesamiento y exportación de datos
Una vez extraído los datos desde el API, según los criterios de búsqueda establecidos, es momento de realizar un procesamiento previo con la finalidad de limpiar un poco los datos y generar nuevos datos en base a los datos obtenidos.

Los procesos incluidos en el procesamiento previos de los datos extraídos son:

- Extracción de emojis de Tweets.
- Cálculo de puntaje de sentimientos negatividad, positividad y neutralidad de emojis.
- Eliminación de emojis de Tweets.
- Eliminar signos de puntuación.
- Tokenización de palabras.
- Eliminación de palabras vacías según idioma indicado (español o ingles).
- Lematización.

Para llevar a cabo cada proceso, se desarrollaron e implantaron un conjunto de funciones las cuales se dividieron en dos partes, funciones para procesamiento de emojis y funciones para procesamientos básicos de textos, dichas funciones se definen a continuación:


In [31]:
###                                           ### 
 # Funciones de procesamiento básico de emojis #
### 

def load_emoji_sentiment(path="Emoji_Sentiment_Data_v1.0"):
    """
        Esta función crea un diccionario de emojis con los scores de negativiad, positivad 
        y neutralidad, entre otros paramretros, de cada emoji del data set que recibe como parametro
        Parámetros:
          path: string
            Ruta del fichero de dataset de emojis
        ----------------------------------------------------------------------------------   
        Return
          emoji_dict : Diccionario
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> emoji_sent_dict = load_emoji_sentiment("Emoji_Sentiment_Data_v1.0.csv")
          >>> emoji_sent_dict["😭"]
              {
                'Negative': 0.4364820846905538,
                'Neutral': 0.22041259500542887,
                'Occurrences': 5526,
                'Position': 0.803351976,
                'Positive': 0.34310532030401736,
                'Unicode block': 'Emoticons',
                'Unicode codepoint': '0x1f62d',
                'Unicode name': 'LOUDLY CRYING FACE'
              }
    """
    # Cargamos el csv de emoji_sentiment
    emoji_sent_df = pd.read_csv(path,sep=",")
    # Calculamos los scores dividiendo el número de emojis negativos y entre el total
    emoji_sent_df["Negative"] = emoji_sent_df["Negative"]/emoji_sent_df["Occurrences"]
    emoji_sent_df["Neutral"] = emoji_sent_df["Neutral"]/emoji_sent_df["Occurrences"]
    emoji_sent_df["Positive"] = emoji_sent_df["Positive"]/emoji_sent_df["Occurrences"]
    # Transformamos a dict
    emoji_sent_df = emoji_sent_df.set_index('Emoji')
    emoji_dict = emoji_sent_df.to_dict(orient="index")
    return emoji_dict

def extract_emojis(text):
    """
      Esta función extrae emojis del texto en formato de lista, si el texto no tiene 
      emojis retorna una lista vacia.
      Parámetros:
        text: string
          Texto con emojis
      ----------------------------------------------------------------------------------   
      Return
        emojis_list : List
      ----------------------------------------------------------------------------------
      Ejemplo:
        >>> extract_emojis("Texto de prueba 🎻 😡")
          ['🎻', '😡']
    """
    extract = Extractor()
    emojis = extract.count_emoji(text, check_first=False)
    emojis_list = [key for key, _ in emojis.most_common()]
    return emojis_list

def get_emoji_sentiment(lista,sent_dict, option = "positive"):
    """
      Esta función calcula el score del sentimento de una lista de emojis, los 
      sentinetimientos puedens ser positivo,negativo o neutral, 
      esta funcion se baja en un diccionario de score de emojis.
      Parámetros:
        lista: List
          lista de emojis
        sent_dict: diccionario
          Diccionarios de score
        option: string
          Sentimento a buscar (positive,negative,neutral), sino se indica parametro 
          retorna sentimiento positivo
      ----------------------------------------------------------------------------------   
      Return
        output : float
      ----------------------------------------------------------------------------------
      Ejemplo:
        >>> get_emoji_sentiment(['🎻', '😡'],emoji_sent_dict, option = "positive")
            0.8042328042328042
    """
    output = 0
    for emoji in lista:
        try:
            if option == "positive":
                output = output + sent_dict[emoji]["Positive"]
            elif option =="negative":
                output = output + sent_dict[emoji]["Negative"]
            elif option =="neutral":
                output = output + sent_dict[emoji]["Neutral"]
        except Exception as e: 
                continue
    return output

def clean_emoji(text):
    """
        Esta función elimina los emojis de un texto. Es util porque podemos quere textos
        sin emejis para mejorar el analisis.

        Parámetros:
          text: string
            Texto con emojis
        ----------------------------------------------------------------------------------   
        Return
          string2 : String
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> clean_emoji("Esto es un texto de prueba 🎻, que contiene emojis 😡")
              "Esto es un texto de prueba  , que contiene emojis " 
    """

    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

###                                           ### 
 # Funciones de procesamiento básico de textos #
###                                           ### 

def quitar_stopwords(tokens,lang="en"): 
    """
        Esta función elimina los stop Word de una lista de tokens.
        las stop Word también conocidas en español 
        como palabras vacías (artículos, pronombres, preposiciones,etc). 
        Esta funcion es para eliminar palabras vacías en idioma ingles. (stopwords.words('english'))

        Parámetros:
            tokens: List
                Lista de tokens
            lang: string
                idioma de las palabras, solo admite ingles y español
        ---------------------------------------------------------------------------------   
        Return
            filtered_sentence : list
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> quitar_stopwords(['I', 'do', 'not', 'think', 'I', 'will', 'ever', 'be', 'a', 'city'])
            ['I', 'think', 'I', 'ever', 'city']
    """
    if lang == "es":
        idioma="spanish"
    else:
        idioma="english"
    stop_words = set(stopwords.words(idioma)) 
    filtered_sentence = [w for w in tokens if not w in stop_words]
    return filtered_sentence

def quitar_puntuacion(tokens):
    """
        Esta función elimina los signos de puntuacion presentes en un texto.
        Retorna una lista de tokens sin signos de puntuación.
        
        Parámetros:
            tokens: List
            Lista de tokens
        ----------------------------------------------------------------------------------   
        Return
            words : list
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> quitar_puntuacion(['I', 'do', 'not', 'think', 'I', 'will', 'ever', 'be', 'a', 'city', '.'])
            ['I', 'do', 'not', 'think', 'I', 'will', 'ever', 'be', 'a', 'city']
    """
    words=[word for word in tokens if word.isalnum()]
    return words

# Lematizar con Spacy
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
def lematizar(tokens,lang,nlp):
    """
        Esta función lematiza una lista de tokens y retorna un string con las palabras lematizadas.
        Es importante tener presente que hay que declarar antes del llamado a la funcion nlp con el modelo
        pre-entrenado en lengaje que se desee lematizar y establecer los parametros que se requieran.
        Esta función es una bajo la librería de Spacy

        INGLES
        nlp = en_core_web_sm.load(disable=['parser', 'ner'])
        ESPAÑOL
        nlp = es_core_news_sm.load(disable=['parser', 'ner'])

        los paquetes pre-entrenados se descargar:
            !python -m spacy download en_core_web_sm #Ingles
            !python -m spacy download es_core_news_sm #Español

        Parámetros:
            tokens: List
                Lista de tokens
            lang: string
                idioma de las palabras, solo admite ingles y español
        ----------------------------------------------------------------------------------   
        Return
            words : string
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> lematizar(['rom', 'roms'])
            "rom rom"
    """
    if lang == "es":
        nlp = es_core_news_sm.load(disable=['parser', 'ner'])
    sentence = " ".join(tokens)
    mytokens = nlp(sentence)
    # Lematizamos los tokens y los convertimos  a minusculas
    mytokens = [ word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Extraemos el text en una string
    return " ".join(mytokens)

def tokenize(texto):
    """
        Esta función tokeniza los registros, se usa el "TweetTokenizer" de 
        NLTK (https://github.com/jaredks/tweetokenize), el cual se usa para tokenizar registros 
        provenientes de la api de twitter, una vez tokenizado el texto retorna una lista de tokens.

        Parámetros:
          texto: string
            Texto a tokenizar
        ----------------------------------------------------------------------------------   
        Return
          tokens_list : list
        ----------------------------------------------------------------------------------
        Ejemplo:
          >>> tokenize("I do not think I will ever be a city")
              ['I', 'do', 'not', 'think', 'I', 'will', 'ever', 'be', 'a', 'city']
    """
    tweet_tokenizer = TweetTokenizer()
    tokens_list = tweet_tokenizer.tokenize(texto)
    return tokens_list


def procesarTexto(dataset,lang="en"):
    """
        Esta función ejecutar  ejecutar todas las funciones de procesamiento básico, tanto de emojis como de texto. 
        Parámetros:
            dataset: dataframe
                Datos a procesar
            lang: string
                Lengauje en el cual se van a procesar los datos
        ----------------------------------------------------------------------------------   
        Return
            Dataset : dataframe
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> procesarTexto(dataset,"es")
      """
    #Cargamos el dataset de score de sentimeintos de emojis y lo almacenamos en un diccionario
    emoji_sent_dict = load_emoji_sentiment("Emoji_Sentiment_Data_v1.0.csv")
    
    #Recorremos los registros para extraer los emoji que contengan cada registro (tweet)
    dataset["emoji_list"] = dataset["TextoOriginal"].apply(lambda x: extract_emojis(x))
    
    #Extraemos los sentimentos positivos,negativos y neutrales de cada emoji que contengan los registros, partiendo del diccionarido de socre de sentimentos de emojis
    dataset["sent_emoji_pos"] = dataset["emoji_list"].apply(lambda x: get_emoji_sentiment(x,emoji_sent_dict,"positive")) #sentimentos positivo
    dataset["sent_emoji_neu"] = dataset["emoji_list"].apply(lambda x: get_emoji_sentiment(x,emoji_sent_dict,"neutral")) #sentimentos neutral
    dataset["sent_emoji_neg"] = dataset["emoji_list"].apply(lambda x: get_emoji_sentiment(x,emoji_sent_dict,"negative")) #sentimentos negativo
    
    # Una vez extareido la información de los emojis, los eliminamos de los registros.
    dataset["TextoProcesado"] = dataset["TextoOriginal"].apply(lambda x: clean_emoji(x))
    
    # Procedemos a tokenizar los datos que tratamotamos anteriormente.
    dataset["TextoProcesado"] = dataset["TextoProcesado"].apply(lambda x: tokenize(x))
    
    # Eliminamos palabras vacias
    dataset["TextoProcesado"] = dataset["TextoProcesado"].apply(lambda x: quitar_stopwords(x))
    
    # Eliminamos los símbolos de puntuación
    dataset["TextoProcesado"] = dataset["TextoProcesado"].apply(lambda x: quitar_puntuacion(x))
    
    # Lematizamos
    dataset["TextoProcesado"] = dataset["TextoProcesado"].apply(lambda x:lematizar(x,lang,nlp))
    return dataset

def CrearTweetDataSet(nombreFichero):
    """
        Pasar los datos dese un fichero .json a un dataframe
        Parámetros:
            nombreFichero: string
                Nombre del fichero a cargar
        ----------------------------------------------------------------------------------   
        Return
            Dataset
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> CrearTweetDataSet('DataTweet.json')
      """
    #Cargamos los tweet que nos descargamos del fichero
    tweetsData=""
    dataset = pd.DataFrame(columns=["TextoOriginal","Regla"])
    with open(nombreFichero) as tweetsFile:
        tweetsData = json.load(tweetsFile)
    for info in tweetsData:
        dataset=dataset.append({"TextoOriginal": info["data"]["text"],"Regla": info["matching_rules"][0]["tag"]},ignore_index=True)
    return dataset


###                                    ### 
 # Funciones de exportacion de fciheros #
###                                    ###
def exportToJSON(dataframe,nombreFichero):
    """
        Exportacion de dataframe en formto json
        Parámetros:
            dataframe: dataframe
                Dataframe de los datos a exportar
            nombreFichero
        ----------------------------------------------------------------------------------   
        Return
            fichero <nombreFichero>.json
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> exportToJSON(leerFicheroDatos('DataTweet.json'),"exportaJSON")
                exportaJSON.json
      """
    dataframe.to_json("{}.json".format(nombreFichero), orient='records')

def exportToEXCEL(dataframe,nombreFichero):
    """
        Exportacion de dataframe en formto excel
        Parámetros:
            dataframe: dataframe
                Dataframe de los datos a exportar
            nombreFichero
        ----------------------------------------------------------------------------------   
        Return
            fichero <nombreFichero>.xlsx
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> exportToEXCEL(leerFicheroDatos('DataTweet.json'),"exportaEXCEL")
                exportaEXCEL.xlsx
    """
    dataframe.to_excel("{}.xlsx".format(nombreFichero), sheet_name='Datos')

def exportToCSV(dataframe,nombreFichero):
    """
        Exportacion de dataframe en formto csv, con separador "|" (barra), 
        de esta forma permitimos comas (,) en le tweets
        Parámetros:
            dataframe: dataframe
                Dataframe de los datos a exportar
            nombreFichero
        ----------------------------------------------------------------------------------   
        Return
            fichero <nombreFichero>.csv
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> exportToCSV(leerFicheroDatos('DataTweet.json'),"exportaCSV")
                exportaCSV.csv
    """
    dataframe.to_csv("{}.csv".format(nombreFichero), sep = "|", index=False)
    
def exportToTXT(dataframe,nombreFichero):
    """
         Exportacion de dataframe en formto txt, con separador "|" (barra), 
        de esta forma permitimos comas (,) en le tweets
        Parámetros:
            dataframe: dataframe
                Dataframe de los datos a exportar
            nombreFichero
        ----------------------------------------------------------------------------------   
        Return
            fichero <nombreFichero>.txt
        ----------------------------------------------------------------------------------
        Ejemplo:
            >>> exportToTXT(leerFicheroDatos('DataTweet.json'),"exportaTXT")
            exportaCSV.txt
    """
    dataframe.to_csv("{}.txt".format(nombreFichero), sep = "|", index=False)

### Ejecucion desde consola

Para este ejemplo de ejecución vamos a realizar un caso de uso, referente a un tema de actualidad.

Se desea crea un dataset sobre acontecimiento reciente en la isla de la palma (España), debido a la erupción del volcán, con la finalidad de analizar la opinión de las personas sobre este hecho desde los tweet omitidos en Twitter.


In [18]:
#Cargamos las credenciales para comunicranos con el API
GuardarToken(token="AAAAAAAAAAAA.....AoDKwsEcHUFLY2wtvRGZ",user="@seijasalomon")

#Definir las como va estar conformada la regla
queryRegla=""
#Lista de palabras que se desea que aparescan en el Tweet
listaPalabrasIncluir1=["volcan","gases toxicos"]
listaPalabrasIncluir2=["volcan","lava"]
listaPalabrasIncluir3=["#LAPALMA","#lapalma","#LaPalma","la Palma","volcan","lava", "vulcanologia", "erupción", "gases toxicos"]
aux1=agruparPalabras(incluirPalabras(listaPalabrasIncluir1,"AND"))
aux2=agruparPalabras(incluirPalabras(listaPalabrasIncluir2,"AND"))
aux3=agruparPalabras(agruparPalabras(incluirPalabras(listaPalabrasIncluir3,"OR")))
#podemos elegir si quieremos buscar Tweet que contengan todos estas palabras o solo una de ellas
queryRegla=agruparPalabras(incluirPalabras([aux1,aux2,aux3],"OR"))

#Definimos el idioma en que se desa que apraxcan los tweet
idioma = tweetIdioma("es")
queryRegla+=" {}".format(idioma)
#Definimos si deseamos que aparzcan o no Retweet, para este caso no nos interesan
retweet=tweetRetweet(False)
queryRegla+=" {}".format(retweet)

# Para definir los criterios de ejecución hay que ecidir entren si se desean conseguir un numero de tweet determinado o por tiempo
# Debido a que estamas probando inicialmnte la regla, vamos a inicar por una configuración por tiempo,
# ya que no sabemos si estas palabras tiene el trafico sufiente y de este modo vamos probando y ajustando la 
# la regla.

CantTweet=0 # no queremos que sea por cantida de Tweets, indicamos cero
Dias=0 # no queremos que sean dias extrayendo de Tweets, indicamos cero

# vamos a empezar por indicarle que extraiga todos los tweet que pueda durante 1 minuto 
# segun los criterios de busqueda establecidos.
Min=1

#Una vez definidos los criterios de busuqeda, vamos a crear las reglas para establecer la comunicacion con el API de Twitter

# Primero Asiganmos numebre a la regla, para identificarla
nombreRegla1 = "Regla Volcan #laPalma"
# Obtenemos todos las reglas que tiene la sesión y las borramos para evitar que se solapen con reglas anteriores.
reglas=obtenerReglas()
borrarReglas(reglas)

#Creamos la regla
crearReglas(rules_value=queryRegla, tag_value=nombreRegla1)

#Una vez definida la regla, procedemos a extraer tweet.
obtenerTweets(cantTweets=CantTweet,minutos=Min,dias=Dias,timeoutMin=10)


In [20]:
# Verificamos si con los parametros establecidos, logramos extraer algunos Tweets,
# por defecto el nombre donde se guardan los tweets descargados se llama DataTweet.json
df=CrearTweetDataSet("DataTweet.json")
pd.set_option('display.max_colwidth', None)
df


,TextoOriginal,Regla
0,"Dicho de otra manera, ¿Qué pasa si lo que la lava se ha llevado por delante era, oficialmente, un almacén de aperos de labranza, pero realmente era una casa completa?",Regla Volcan #laPalma
1,Qué pasa si legalmente tienes esto: https://t.co/I7pz3swCQ6,Regla Volcan #laPalma
2,Vivimos en un pais bananero con unos gobernantes inútiles e ignorantes. https://t.co/MyHZ4VbIc6,Regla Volcan #laPalma
3,"@MartaArmengou @old_eyre @jlrebordinos @sansebastianfes El problema es la importancia que le dais a ganar premios. Creo que hay que reivindicar a las grandes autoras, no a las premiadas. ¿A quién le importa hoy la Palma a Jane Campion? Lo que critica ese crítico es la construcción de una cultura materialista en nombre de lo femenino.",Regla Volcan #laPalma
4,De locos lo de @kikeperezcomico 🎩 \nTodos con la #islabonita🤍 https://t.co/1A47y0U918,Regla Volcan #laPalma
5,"@Don_Richal @AminaCroc Hay ferrys que desde La Palma tardan 3 o 4 horas en llegar a los Cristianos en Tenerife...y alli si hay suficientes hoteles para poder llevar a los evacuados. Si no lo han hecho es porque son unos sinvergüenzas, alojan a los moros de las pateras y a los vecinos de isla tirados",Regla Volcan #laPalma
6,Así se ve desde el aire convecino de archipiélago \nLa isla de la Palma. Y lo redondo es la caldera .de Taburiente. https://t.co/Zd3vTJXbBl,Regla Volcan #laPalma
7,"@mapriego08 @floperino No voy a mentir, hay una parte muy grande dentro de mi que quiere ir, ahora bien, cualquier persona con dos dedos de frente sabrá que no es buena idea, básicamente lo que dice escarchado, es trágico pero bonito a la vez",Regla Volcan #laPalma
8,@EspejoPublico Que visionarios,Regla Volcan #laPalma
9,"@javiro_ser Lo que ja ocurrido es una catástrofe natural,\nLlamas a la responsabilidad informativa, cuando los mediios manipulan lo que quieren, y casi siempre ocultan la verdad de las noticias",Regla Volcan #laPalma


Podemos observar que en un minuto se descargar aron 30 Tweets los cuales cumplen con las reglas establecidas, algunos no harán referente a los temas establecidos, pero recordemos que es el api de Twitter que realiza la indexación de cada tweet.

Una vez inspeccionados los datos, si vemos que hay datos que no pertenecen al contexto que necesitamos, probamos con eleccionar nuevas palabras u omitir a esos usuarios o palabras que no queremos que aparezcan en nuestra búsqueda.

Ahora mantendremos las palabras anteriores, pero añadiremos otra regla, para eso definimos todos los criterios nuevamente como en la sección anterior.

In [19]:
#Definir las como va estar conformada la regla
queryRegla2=""
#Lista de palabras que se desea que aparescan en el Tweet
listaPalabrasIncluirR2=["volcan","Cumbre","Vieja"]
queryRegla2=agruparPalabras(incluirPalabras(listaPalabrasIncluirR2,"AND"))
#podemos elegir si quieremos buscar Tweet que contengan todos estas palabras o solo una de ellas


#Definimos el idioma en que se desa que apraxcan los tweet
idioma = tweetIdioma("es")
queryRegla2+=" {}".format(idioma)
#Definimos si deseamos que aparzcan o no Retweet, para este caso no nos interesan
retweet=tweetRetweet(False)
queryRegla2+=" {}".format(retweet)

# Para definir los criterios de ejecución hay que ecidir entren si se desean conseguir un numero de tweet determinado o por tiempo
# Debido a que estamas probando inicialmnte la regla, vamos a inicar por una configuración por tiempo,
# ya que no sabemos si estas palabras tiene el trafico sufiente y de este modo vamos probando y ajustando la 
# la regla.

CantTweet=0 # no queremos que sea por cantida de Tweets, indicamos cero
Dias=0 # no queremos que sean dias extrayendo de Tweets, indicamos cero

# vamos a empezar por indicarle que extraiga todos los tweet que pueda durante 1 minuto 
# segun los criterios de busqueda establecidos.
Min=1

#Una vez definidos los criterios de busuqeda, vamos a crear las reglas para establecer la comunicacion con el API de Twitter

# Primero Asiganmos numebre a la regla, para identificarla
nombreReglar2 = "Regla Volcan Cumbre vieja"
# Obtenemos todos las reglas que tiene la sesión y las borramos para evitar que se solapen con reglas anteriores.
#reglas=obtenerReglas()
#borrarReglas(reglas)

# En este paso ya tenemos cargada la regla anterior solo queremos añadir una nueva regla por lo cual omitimos el borrado de la regla anterior ya que
# la queremos conturnar usando.

crearReglas(rules_value=queryRegla2, tag_value=nombreReglar2)

#Ejecutamos nuevamente la extraccion
obtenerTweets(cantTweets=CantTweet,minutos=Min,dias=Dias,timeoutMin=10)

True

In [20]:
# Verificamos si con los parametros establecidos, logramos extraer algunos Tweets,
# por defecto el nombre donde se guardan los tweets descargados se llama DataTweet.json
df=CrearTweetDataSet("DataTweet.json")
pd.set_option('display.max_colwidth', None)
dfS

,TextoOriginal,Regla
0,"Dicho de otra manera, ¿Qué pasa si lo que la lava se ha llevado por delante era, oficialmente, un almacén de aperos de labranza, pero realmente era una casa completa?",Regla Volcan #laPalma
1,Qué pasa si legalmente tienes esto: https://t.co/I7pz3swCQ6,Regla Volcan #laPalma
2,Vivimos en un pais bananero con unos gobernantes inútiles e ignorantes. https://t.co/MyHZ4VbIc6,Regla Volcan #laPalma
3,"@MartaArmengou @old_eyre @jlrebordinos @sansebastianfes El problema es la importancia que le dais a ganar premios. Creo que hay que reivindicar a las grandes autoras, no a las premiadas. ¿A quién le importa hoy la Palma a Jane Campion? Lo que critica ese crítico es la construcción de una cultura materialista en nombre de lo femenino.",Regla Volcan #laPalma
4,De locos lo de @kikeperezcomico 🎩 \nTodos con la #islabonita🤍 https://t.co/1A47y0U918,Regla Volcan #laPalma
5,"@Don_Richal @AminaCroc Hay ferrys que desde La Palma tardan 3 o 4 horas en llegar a los Cristianos en Tenerife...y alli si hay suficientes hoteles para poder llevar a los evacuados. Si no lo han hecho es porque son unos sinvergüenzas, alojan a los moros de las pateras y a los vecinos de isla tirados",Regla Volcan #laPalma
6,Así se ve desde el aire convecino de archipiélago \nLa isla de la Palma. Y lo redondo es la caldera .de Taburiente. https://t.co/Zd3vTJXbBl,Regla Volcan #laPalma
7,"@mapriego08 @floperino No voy a mentir, hay una parte muy grande dentro de mi que quiere ir, ahora bien, cualquier persona con dos dedos de frente sabrá que no es buena idea, básicamente lo que dice escarchado, es trágico pero bonito a la vez",Regla Volcan #laPalma
8,@EspejoPublico Que visionarios,Regla Volcan #laPalma
9,"@javiro_ser Lo que ja ocurrido es una catástrofe natural,\nLlamas a la responsabilidad informativa, cuando los mediios manipulan lo que quieren, y casi siempre ocultan la verdad de las noticias",Regla Volcan #laPalma


In [24]:
# 3Ejecutamos nuevamente la extraccion pero ahora indicando la cantidad de Tweet 
# y el tiempo de temeout en caso de que no encuentre nada
Dias=0
Min=0
obtenerTweets(cantTweets=1000,minutos=Min,dias=Dias,timeoutMin=25)

#Una vez finalizado el proceso podemos observamos cuando tweet logramos descargar

False

In [26]:
# Verificamos nuevamente si con los parametros establecidos, logramos extraer algunos Tweets,
# por defecto el nombre donde se guardan los tweets descargados se llama DataTweet.json
df=CrearTweetDataSet("DataTweet.json")
pd.set_option('display.max_colwidth', None)
df

,TextoOriginal,Regla
0,Qué bueno esto 👇 https://t.co/9rwxZinJCY,Regla Volcan #laPalma
1,"Se está donando la camiseta del Sevilla FC firmada por toda la plantilla, en el directo de Twitch de @kikeperezcomico \nDonación del gran @RarrochaR \nPara ayudar a La Palma",Regla Volcan #laPalma
2,El aeropuerto de la Palma reabre aunque los vuelos se cancelaron debido a que continúa la erupción del volcán de la isla https://t.co/dc0W5AGSpb,Regla Volcan #laPalma
3,@TNTSportsCL 2 mala calidad de imagen y ahora suman a la gb a relato y comentario. https://t.co/u3ilgXDYe8,Regla Volcan #laPalma
4,@isolar75 @rtve Eso te pasa por poner RTVE,Regla Volcan #laPalma
...,...,...
650,@WnaRota @ClimaPeruMundo @dw_espanol 5mentarios,Regla Volcan #laPalma
651,@Ricardo13625223 Pero no veras a este gobierno agradecerlo pq el Pedrito mentiras no quiere ejercito.,Regla Volcan #laPalma
652,"@PalmaCorrupcion @JuanyGar63 Madre mía, espero se indemnice a quien lo está perdiendo todo!",Regla Volcan #laPalma
653,"de dondeee ostias te has escapado? Lo único que por desgracia a sido provocado aquí fue tu parto, que pena @Rutenius1 https://t.co/9efgbyIcwO",Regla Volcan #laPalma


Como vemos que las reglas definidas estan permitiendo la extraccion de datos, vamos a realizar una solcitud de una cantidad mayor de datos, por ejemplo unos 1000 tweets y analizar los resultados.

In [27]:
# Verificamos nuevamente si con los parametros establecidos, logramos extraer algunos Tweets,
# por defecto el nombre donde se guardan los tweets descargados se llama DataTweet.json
df=CrearTweetDataSet("DataTweet.json")
pd.set_option('display.max_colwidth', None)
df

,TextoOriginal,Regla
0,Qué bueno esto 👇 https://t.co/9rwxZinJCY,Regla Volcan #laPalma
1,"Se está donando la camiseta del Sevilla FC firmada por toda la plantilla, en el directo de Twitch de @kikeperezcomico \nDonación del gran @RarrochaR \nPara ayudar a La Palma",Regla Volcan #laPalma
2,El aeropuerto de la Palma reabre aunque los vuelos se cancelaron debido a que continúa la erupción del volcán de la isla https://t.co/dc0W5AGSpb,Regla Volcan #laPalma
3,@TNTSportsCL 2 mala calidad de imagen y ahora suman a la gb a relato y comentario. https://t.co/u3ilgXDYe8,Regla Volcan #laPalma
4,@isolar75 @rtve Eso te pasa por poner RTVE,Regla Volcan #laPalma
...,...,...
650,@WnaRota @ClimaPeruMundo @dw_espanol 5mentarios,Regla Volcan #laPalma
651,@Ricardo13625223 Pero no veras a este gobierno agradecerlo pq el Pedrito mentiras no quiere ejercito.,Regla Volcan #laPalma
652,"@PalmaCorrupcion @JuanyGar63 Madre mía, espero se indemnice a quien lo está perdiendo todo!",Regla Volcan #laPalma
653,"de dondeee ostias te has escapado? Lo único que por desgracia a sido provocado aquí fue tu parto, que pena @Rutenius1 https://t.co/9efgbyIcwO",Regla Volcan #laPalma


In [30]:
# Como podemos Observar Obtuvimos al rededor de 600 tweet en 25 min, la gran mayoria referente al tema de interes.
# El siguiente paso, es el procesado preveio y la exprtanción del data set.

#Proecesamos los datos extraidos
dfProcesado=procesarTexto(df,"es")
#Exportamos los datos en diferentes formatos
exportToEXCEL(dfProcesado,"Exportacion/Datos-laPalma")
exportToCSV(dfProcesado,"Exportacion/Datos-laPalma")
exportToTXT(dfProcesado,"Exportacion/Datos-laPalma")
exportToJSON(dfProcesado,"Exportacion/Datos-laPalma")

### Interfaz Gráfica
El proceso de extracción de datos, en tiempo real, desde el API de Twitter se puede realizar completamente desde el script del júpiter notebook, implementando las funciones explicitadas anteriormente, sim embargo se desarrolló una pequeña interfaz para facilitar el proceso de interacción entre la herramienta y el usuario.

In [55]:
FONT_AWESOME = "https://use.fontawesome.com/releases/v5.7.2/css/all.css"
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP,FONT_AWESOME], meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

PLOTLY_LOGO = app.get_asset_url('logoHorizontal.png')

Navbar = dbc.Navbar(
    [
        html.A(
            dbc.Row([
                dbc.Col(html.Img(src=PLOTLY_LOGO, height="55rem"),className="p-1"),
                dbc.Col(dbc.NavbarBrand("", className="ml-2")),
            ],align="Center",no_gutters=True,),
            href=""),
        dbc.Col([
            dbc.Button(" ",id="btnCredenciales", color="primary", className="fas fa-solid fa-lock",n_clicks=0),
            dbc.Modal(
            [
                dbc.ModalHeader("Token Auntenticación Twitter"),
                dbc.ModalBody([
                    dbc.Input(id="tokenUser", placeholder="Ingrese usuario twitter", className="m-2"),
                    dbc.Input(id="token", placeholder="Ingrese Token de autenticación",className="m-2"),
                    html.Div(id='TokenValido',className="m-2 text-center")
                ]),
                dbc.ModalFooter([
                    dbc.Row([
                        dbc.Button("Cerrar", id="BtnCerrar", color="danger",className="text-center ml-2", n_clicks=0),
                        dbc.Button("Test", id="BtnTest", color="info",className="text-center ml-2", n_clicks=0),
                        dbc.Button("Guardar", id="BtnGuardar", color="primary",className="text-center ml-2", n_clicks=0),
                        
                    ])                    
                ]),
            ],
            id="modal",
            is_open=False,
        ),
        ],className="text-right"),
    ],
    id="Navbar",
    color="dark",
    dark=True,
)
@app.callback(
    Output("modal", "is_open"),
    [Input("btnCredenciales", "n_clicks"), Input("BtnCerrar", "n_clicks"),Input("BtnGuardar", "n_clicks")],
    [State("modal", "is_open"),State("token", "value")],
)
def toggle_modal(btnCredenciales,BtnCerrar,BtnGuardar,is_open,token):
    if BtnGuardar > 0:
        if len(token):
            GuardarToken(token)
        return not is_open
    elif btnCredenciales > 0 or BtnGuardar > 0:
        return not is_open

@app.callback(
    Output("TokenValido", "children"),
    [Input("BtnTest", "n_clicks")],
    [State("token", "value"),State("tokenUser", "value")],
)
def validarToken(BtnTest,token="",tokenUser=""):
    if BtnTest > 0:
        if token !=None and tokenUser !=None and len(token) > 0 and len(tokenUser) > 0:
            GuardarToken(token,tokenUser)
            if "@" in tokenUser:
                tokenUser = tokenUser.replace("@","")
            if validarUsuario(tokenUser):
                return "Token valido."
            else:
                return "Token no valido."
        return "Introducir los datos solicitados."
            
#Componenetes parametros generales

# idiomo
idioma = dbc.Col([
            dbc.Label("Idioma:",html_for="lengtweet",width=12, className="p-0", style={"fontSize":"16px"}),
            dbc.RadioItems(options=[{"label": "Ingles", "value": "en"},{"label": "Español", "value": "es"}],
                value="en",id="lengtweet",inline=True)
            ],width=6)

# retweet
retweet = dbc.Col([
            dbc.Label("", html_for="retweet", width=12,className="p-0"),
            dbc.Checklist(options=[{"label": "¿Retweets?", "value": True}],value=False,id="retweet",inline=True,switch=True)
            ],width=6)

#Periodicidad
SelectorEjecucion = dbc.Col([
                dbc.Label("Formas de ejecucíon:", html_for="ejecucion"),
                dcc.Dropdown(
                    options=[
                        {"label": "Cantidad de Tweets", "value": 1},
                        {"label": "Tiempo de ejecución", "value": 2},
                    ],id="ejecucion",placeholder="Seleccionar")
                ],width=12,style={"paddingLeft": "15px", "marginTop": "15px","marginBottom":"15px"})
@app.callback(
    [Output('CantTweetCol', 'style'),Output('TiempoTweetCol', 'style')],
    Input('ejecucion', 'value')
)
def MostrarTiempoEjecucion(value):
    """
        Esta función muetra la forma de periodicidad que se desea ejecutar
        Parámetros:
                value: integer
                    Indice asociado  la opcion selecionada (cantidad de tweets=1, timepo ejecucion = 2)
            ----------------------------------------------------------------------------------   
            Return
                palabras :stylo para mostrar o ocultar el campo
            ----------------------------------------------------------------------------------
            Ejemplo:
                >>> MostrarTiempoEjecucion(1)
                    [{"display": "block"},{"display": "none"}]
    """
    if value == 1:
        return [{"display": "block"},{"display": "none"}]
    elif value == 2:
        return [{"display": "none"},{"display": "block"}]
    else:
        return [{"display": "none"},{"display": "none"}]
        

#Cantidad de tweet
cantidadTweet =dbc.Col([
                    dbc.Label("Cantidad de Tweets:", html_for="CantTweet", width=12,className="p-0"),
                    dbc.Input(type="number", id="CantTweet", placeholder="0",required=False,style={"fontSize":"12px"}),
                ],width=12,style={"display": "none","marginTop": "10px","marginBottom":"10px"},id="CantTweetCol")

#Ejecución por tiempo
TiempoTweet = dbc.Col([
                    dbc.Label("Tiempo de ejecución:", html_for="TiempoTweet", width=12,className="p-0"),
                    dbc.Row([
                        dbc.Col([dbc.Input(type="number", id="TiempoTweetDias", placeholder="Días",required=False,style={"fontSize":"12px"})],width=4),
                        dbc.Col([dbc.Input(type="number", id="TiempoTweetHoras", placeholder="Horas",required=False,style={"fontSize":"12px"})],width=4),
                        dbc.Col([dbc.Input(type="number", id="TiempoTweetMin", placeholder="Min",required=False,style={"fontSize":"12px"})],width=4),
                    ],id="TiempoTweet")
                ],width=12, style={"display": "none","marginTop": "10px","marginBottom":"10px"},id="TiempoTweetCol")

#Formularios parametros generales 
formularioParamGenerales= dbc.FormGroup([
                            dbc.Row([idioma,retweet,SelectorEjecucion,cantidadTweet,TiempoTweet],className="m-3")
                          ])

CardformularioParamGenerales =dbc.Card([
                dbc.CardHeader(
                    html.H2(
                        dbc.Button(f"Parametros Generales",color="link",id={'type': 'btnReglaToggle','index': 54321},n_clicks=0,))),
                        dbc.Collapse(formularioParamGenerales,id={'type': 'collapse','index': 54321},is_open=False,),
            ])

form = dbc.Form([CardformularioParamGenerales],)

#Reglas
# Formulario de reglas

def CrearForm(i):
    """
        Esta función define la estructura interna del formulario que comforma una regla.
        Parámetros:
                i: integer
                Indice asociado a la regla recien creada.
            ----------------------------------------------------------------------------------   
            Return
                palabras : Estructura HTML (string)
            ----------------------------------------------------------------------------------
            Ejemplo:
                >>> CrearForm(0)
                    la salida es el HTML resultante, sutituyendo el valir de i pasado como parametro.
    """
    palabras = dbc.Form([
        dbc.FormGroup([
            dbc.Label("Nombre regla:", html_for={'type': 'NombreRegla','index': i}, width=12),
            dbc.Col(
                dbc.Input(type="text",id={'type': 'NombreRegla','index': i}, 
                    placeholder="Ingrese nombre de la regla",style={"fontSize":"12px"}
                ),width=12
            ),
            dbc.Label("Palabras a incluir:", html_for={'type': 'palabrasIncluidas','index': i}, width=12),
            dbc.Col(
                dbc.Input(type="text", 
                          id={'type': 'palabrasIncluidas','index': i}, 
                          placeholder="Ingrese lista de palabras separadas por coma ','",style={"fontSize":"12px"}
                ),
                width=12),
            dbc.Label("Palabras a Excluir:", html_for={'type': 'palabrasExcluidas','index': i}, width=12),
            dbc.Col(
                dbc.Input(type="text",id={'type': 'palabrasExcluidas','index': i},
                          placeholder="Ingrese lista de palabras separadas por coma ','",
                          style={"fontSize":"12px"}
                ),width=12,
            ),
            dbc.Row([
                dbc.Button( "Test", id={'type': 'BtnProbarRegla','index': i},color="info", className="mr-2",n_clicks=0),
                dbc.Button( "Borrar", id={'type': 'BtnBorrarRegla','index': i},color="danger", className="mr-2",n_clicks=0)
            ],className="p-2", style={"marginLeft":"22rem"}),
            dbc.Col(html.Div(id={'type': 'SalidaRegla','index': i}),width=12)
        ],row=True),
    ],className="p-2")
    return palabras

# callback crearRegla #
# Permite probara cada una de las reglas haciendo click en el boton BtnProbarRegla
# que se encuentra dentro del formulario de cada regla, la finalidad es evaluar si 
# la regla funiciona como es de esperar, en casa de que no podemos reajustar dicha regla
# antes de procesar todo el conjunto para la creación del dataset final.

@app.callback(
    Output({'type': 'SalidaRegla', 'index': MATCH}, 'children'),
    Input({'type': 'BtnProbarRegla', 'index': MATCH}, "n_clicks"),
    State({'type': 'NombreRegla', 'index': MATCH}, "value"),
    State({'type': 'palabrasIncluidas', 'index': MATCH}, "value"),
    State({'type': 'palabrasExcluidas', 'index': MATCH}, "value"),
)
def crearRegla(n_clicks,mombreRegla,palabrasIncluidas,palabrasExcluidas,lengtweet="en",CantTweet=10,retweet=False):
    if n_clicks > 0:
        consulta=""
        if palabrasIncluidas !=None :
            #Listas de palabras a buscar, es valido buscar por hashtag (#)
            listaPalabras=palabrasIncluidas.split(",")
            if len(listaPalabras) > 0 and listaPalabras !=None:
                consulta=incluirPalabras(listaPalabras,"OR")

        if palabrasExcluidas !=None :
            # Lista de palabras a excluir
            listaPalabrasExcluidas=palabrasExcluidas.split(",")
            if len(listaPalabrasExcluidas) > 0:
                consulta+=" {}".format(excluirPalabras(listaPalabrasExcluidas))

            consulta+=" {}".format(tweetIdioma(lengtweet))
            consulta+=" {}".format(tweetRetweet(retweet))
            reglas=obtenerReglas()
            borrarReglas(reglas)
            crearReglas(rules_value=consulta, tag_value=mombreRegla)
            obtenerTweets(cantTweets=CantTweet,minutos=0,dias=0)

            return "Finalizado"

### Boton agregarRegla reubicar
#AgregarReglas = html.Div([
#    dbc.Button( "Añadir Regla", id="AddRegla", className="mr-2",n_clicks=0),
#    html.Div(id='accordion-container', children=[]),
#])

def CrearListasReglas(i):
    """
        Esta función define la estructura html para cada regla.
        Parámetros:
                i: integer
                Indice asociado a la regla recine creada.
            ----------------------------------------------------------------------------------   
            Return
                Estructura HTML (string)
            ----------------------------------------------------------------------------------
            Ejemplo:
                >>> CrearListasReglas(0)
                    dbc.Card([
                        dbc.CardHeader(
                            html.H2(
                                dbc.Button(f"Regla #1",color="link",id={'type': 'btnReglaToggle','index': 1},n_clicks=0,))),
                                dbc.Collapse(CrearForm(1),id={'type': 'collapse','index': 1},is_open=False,),])
    """
    return dbc.Card([
                dbc.CardHeader(
                    html.H2(
                        dbc.Button(f"Regla {i+1}",color="link",id={'type': 'btnReglaToggle','index': i},n_clicks=0,))),
                        dbc.Collapse(CrearForm(i),id={'type': 'collapse','index': i},is_open=False,),
            ])

# callback AddReglaAccordion #
# Añade un formulario de reglas nuevo.

@app.callback(
    Output('accordion-container', 'children'),
    Input('AddRegla', 'n_clicks'),
    State('accordion-container', 'children'))
def AddReglaAccordion(n_clicks, children):
    new_dropdown = CrearListasReglas(n_clicks)
    children.append(new_dropdown)
    return children

# callback AbrirCerrarAccordion #
# Funcionalidad netamente estetica para poder abrir o cerrar 
# las regalas creadas.

@app.callback(
    Output({'type': 'collapse', 'index': MATCH}, "is_open"),
    Input({'type': 'btnReglaToggle', 'index': MATCH}, "n_clicks"),
    State({'type': 'collapse', 'index': MATCH}, "is_open"),
)
def AbrirCerrarAccordion(n_clicks,is_open):
    if n_clicks > 0:
        return not is_open

# callback ProcesarRegla #
# Toma todas reglas ya testadas, la procesa en el api de twitter,
# para porteriormente generar el dataset segun la data disponibles en este
# momento en twitter.

@app.callback(
    [Output('tabla-container','children'),Output('cargando','style')],
    Input("BtnProcesar", "n_clicks"),
    State({'type': 'NombreRegla', 'index': ALL}, "id"),
    State({'type': 'NombreRegla', 'index': ALL}, "value"),
    State({'type': 'palabrasIncluidas', 'index': ALL}, "value"),
    State({'type': 'palabrasExcluidas', 'index': ALL}, "value"),
    State("lengtweet", "value"),
    State("retweet", "value"),
    State("CantTweet", "value"),
    State("TiempoTweetDias", "value"),
    State("TiempoTweetHoras", "value"),
    State("TiempoTweetMin", "value"),
    
)
def ProcesarRegla(n_clicks,nombreReglaID,nombreRegla,palabrasIncluidas,palabrasExcluidas,lengtweet,retweet,CantTweet=0,TiempoTweetDias=0,TiempoTweetHoras=0,TiempoTweetMin=0):
    if n_clicks > 0:
        
        if retweet == [True]:
            retweet=True
        elif retweet == []:
            retweet=False
        
        reglas=obtenerReglas()
        borrarReglas(reglas)
        for regla in nombreReglaID:
            index=regla['index']
            consulta=""
            if palabrasIncluidas[index] !=None :
                #Listas de palabras a buscar, es valido buscar por hashtag (#)
                listaPalabras=palabrasIncluidas[index].split(",")
                if len(listaPalabras) > 0:
                    consulta=incluirPalabras(listaPalabras,"OR")

                # de esta forma se determinana las frases y van separadas por coma en caso de que se quieran incluir
                #listafrases=[w for w in listaPalabrasExcluidas if " " in w]

            if palabrasExcluidas[index] !=None :
                # Lista de palabras a excluir
                listaPalabrasExcluidas=palabrasExcluidas[index].split(",")
                if len(listaPalabrasExcluidas) > 0:
                    consulta+=" {}".format(excluirPalabras(listaPalabrasExcluidas))
            
            consulta+=" {}".format(tweetIdioma(lengtweet))
            consulta+=" {}".format(tweetRetweet(retweet))
            crearReglas(rules_value=consulta, tag_value=nombreRegla[index])
        
        if TiempoTweetHoras!= None and TiempoTweetHoras > 0 :
            TiempoTweetMin=TiempoTweetSec*60
        else:
            TiempoTweetHoras=0
            
        if TiempoTweetMin == None:
              TiempoTweetMin=0
                
        if TiempoTweetDias == None:
              TiempoTweetDias=0
        
        if CantTweet == None:
            Cantidad = 0
                
        obtenerTweets(cantTweets=CantTweet,minutos=TiempoTweetMin,dias=TiempoTweetDias,timeoutMin=10)
        table = dbc.Table.from_dataframe(CrearTweetDataSet('DataTweet.json'), striped=True, 
                                         bordered=True, 
                                         hover=True,
                                         responsive=True,
                                         className="m-3")
        return [table,{"display": "none"}]
    return["",style_cargando]
                 
    
##
## CONTENT
##

@app.callback(
    Output("cargando", "children"),
    [Input("BtnProcesar", "n_clicks")]
)
def cargandoData(BtnProcesar):
    if BtnProcesar > 0:
        cargando=dbc.Spinner(color="primary",spinner_style={"width": "5rem", "height": "5rem"})
        return cargando

    cargando=dbc.Spinner(color="primary",spinner_style={"width": "5rem", "height": "5rem"})
#Tabla previsualizacion
#table = dbc.Table.from_dataframe(CrearTweetDataSet('DataTweet.json'), striped=True, bordered=True, hover=True,responsive=True,className="m-3")
#table = dbc.Table.from_dataframe(id="tablaDatos", striped=True, bordered=True, hover=True,responsive=True,className="m-3")
reglas = html.Div(id='accordion-container', children=[],className="")
Btn=dbc.Row([
        dbc.Col([dbc.Button( "Ejecutar", id="BtnProcesar",color="primary", className="mr-2",n_clicks=0,block=True)]),
        dbc.Col([dbc.Button( "Añadir Regla", id="AddRegla", className="mr-2",n_clicks=0,block=True),]),
        dbc.Col([dbc.Button( "Exportar Ficheros", id="BtnExportar", className="mr-2",n_clicks=0,block=True),]),
    ],className="m-3")

@app.callback(
    Output("DivExportar", "children"),
    [Input("BtnExportar", "n_clicks")]
)
def cargandoData(BtnExportar):
    if BtnExportar > 0:
        df=CrearTweetDataSet("DataTweet.json")
        dfProcesado=procesarTexto(df,"es")
        #Exportamos los datos en diferentes formatos
        exportToEXCEL(dfProcesado,"Exportacion/Datos")
        exportToCSV(dfProcesado,"Exportacion/Datos")
        exportToTXT(dfProcesado,"Exportacion/Datos")
        exportToJSON(dfProcesado,"Exportacion/Datos")
        return "Ficheros exportados correctamente"

style_cargando ={
    "height": "100vh",
    "display": "flex",
    "justifyContent": "center",
    "alignItems": "center",    
}    

content = dbc.Row([
        dbc.Col([form,reglas,Btn],className="m-0 p-0",width=4,style={"backgroundColor":"#f8f8f8", }),
        dbc.Col([html.Div(id="DivExportar"),html.Div(id='cargando',style=style_cargando),html.Div(id="tabla-container",style={"height":"100vh", "overflowY": "auto","overflowX": "hidden"})],width=8)
    ],id="Content",className="container-fluid", style={"height":"100vh"})

##
## MAIN HTML
##

body = dbc.Container([Navbar,content], className="col-12 p-0",style={"height":"100%", })
app.layout=body

# Run app and display result inline in the notebook
#app.run_server(mode='inline')
app.run_server(mode='external',debug=False)

Dash app running on http://127.0.0.1:8050/
